# Women Harassment Risk Predictor - ML Model Training

This notebook contains the complete pipeline for training a machine learning model to predict harassment risk levels based on various factors.

## Pipeline Steps:
1. Import Required Libraries
2. Load and Explore Dataset
3. Data Cleaning and Preprocessing
4. Feature Engineering
5. Model Training and Comparison
6. Model Evaluation
7. Hyperparameter Tuning
8. Save Final Model

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Machine Learning models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Model evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

## 2. Load and Explore Dataset

In [ ]:
# Load the dataset
data_path = "data/women_risk.csv"
df = pd.read_csv(data_path)

print("="*60)
print("DATASET LOADED SUCCESSFULLY")
print("="*60)
print(f"\nDataset Shape: {df.shape}")
print(f"Number of Rows: {df.shape[0]}")
print(f"Number of Columns: {df.shape[1]}")
print("\n" + "="*60)

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
print(df.head())

In [ ]:
# Dataset information
print("\nDataset Info:")
print(df.info())
print("\n" + "="*60)

# Statistical summary
print("\nStatistical Summary:")
print(df.describe())
print("\n" + "="*60)

# Column names
print("\nColumn Names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

## 3. Data Cleaning and Preprocessing

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found!")

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Remove Timestamp column if it exists
if 'Timestamp' in df.columns:
    df = df.drop('Timestamp', axis=1)
    print("\n✅ Timestamp column removed")

# Drop any rows with missing values
if df.isnull().sum().sum() > 0:
    df = df.dropna()
    print(f"✅ Rows with missing values removed")

# Remove duplicates
if duplicates > 0:
    df = df.drop_duplicates()
    print(f"✅ Duplicate rows removed")

print(f"\nFinal dataset shape: {df.shape}")

In [ ]:
# Encode categorical variables using Label Encoding
print("="*60)
print("ENCODING CATEGORICAL VARIABLES")
print("="*60)

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nCategorical columns found: {len(categorical_cols)}")

label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"✅ Encoded: {col}")

# Save label encoders
os.makedirs('models', exist_ok=True)
joblib.dump(label_encoders, 'models/label_encoders.pkl')
print("\n✅ Label encoders saved to 'models/label_encoders.pkl'")

## 4. Feature Engineering and Target Creation

In [ ]:
# Create binary target variable based on risk level (last column)
# Assuming the last column contains risk level information
target_col = df.columns[-1]
print(f"Target column: {target_col}")
print(f"\nUnique values in target: {df[target_col].unique()}")
print(f"\nTarget distribution:")
print(df[target_col].value_counts())

# Create binary risk variable: 1 for high/moderate risk, 0 for low/no risk
# Adjust threshold based on your data (e.g., >= 2 means high risk)
df['risk'] = (df[target_col] >= df[target_col].median()).astype(int)

print(f"\nBinary risk distribution:")
print(df['risk'].value_counts())
print(f"\nRisk percentage:")
print(df['risk'].value_counts(normalize=True) * 100)

In [ ]:
# Correlation analysis
plt.figure(figsize=(14, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Correlation with target variable
print("\nCorrelation with risk (target):")
target_corr = correlation_matrix['risk'].sort_values(ascending=False)
print(target_corr)

## 5. Split Data into Training and Testing Sets

In [ ]:
# Separate features and target
X = df.drop(['risk', target_col], axis=1)
y = df['risk']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples ({80}%)")
print(f"Testing set: {X_test.shape[0]} samples ({20}%)")

print(f"\nTraining set target distribution:")
print(y_train.value_counts())
print(f"\nTesting set target distribution:")
print(y_test.value_counts())

In [ ]:
# Feature scaling using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for clarity
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Save the scaler
joblib.dump(scaler, 'models/scaler.pkl')
print("✅ Feature scaling completed")
print("✅ Scaler saved to 'models/scaler.pkl'")

## 6. Train Multiple Classification Models

In [ ]:
# Define multiple models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

print("="*70)
print("TRAINING MULTIPLE MODELS WITH 5-FOLD CROSS-VALIDATION")
print("="*70)
print(f"\n{'Model':<25} {'Mean Accuracy':<15} {'Std Dev':<10}")
print("-"*70)

results = {}

for name, model in models.items():
    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    
    results[name] = {
        'model': model,
        'mean_accuracy': cv_scores.mean(),
        'std_dev': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"{name:<25} {cv_scores.mean():<15.4f} {cv_scores.std():<10.4f}")

print("\n" + "="*70)

In [ ]:
# Visualize model comparison
plt.figure(figsize=(12, 6))
model_names = list(results.keys())
mean_accuracies = [results[name]['mean_accuracy'] for name in model_names]
std_devs = [results[name]['std_dev'] for name in model_names]

plt.bar(model_names, mean_accuracies, yerr=std_devs, capsize=5, 
        color='skyblue', edgecolor='navy', alpha=0.7)
plt.xlabel('Model', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title('Model Comparison (5-Fold Cross-Validation)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.ylim([min(mean_accuracies) - 0.05, 1.0])
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Find best model
best_model_name = max(results, key=lambda x: results[x]['mean_accuracy'])
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Mean Accuracy: {results[best_model_name]['mean_accuracy']:.4f}")

## 7. Train and Evaluate Best Model on Test Set

In [ ]:
# Train best model on full training set
best_model = results[best_model_name]['model']
best_model.fit(X_train_scaled, y_train)

# Make predictions on test set
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("="*60)
print("MODEL PERFORMANCE ON TEST SET")
print("="*60)
print(f"\nModel: {best_model_name}")
print(f"\nAccuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("\n" + "="*60)

## 8. Visualize Model Performance

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Low Risk', 'High Risk'],
            yticklabels=['Low Risk', 'High Risk'])
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

print("Confusion Matrix:")
print(cm)

In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'ROC Curve - {best_model_name}', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Classification Report
print("Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Low Risk', 'High Risk']))
print("="*60)

## 9. Make Sample Predictions

In [ ]:
# Make predictions on a few test samples
n_samples = 5
sample_indices = np.random.choice(X_test.index, n_samples, replace=False)

print("Sample Predictions:")
print("="*80)

for idx in sample_indices:
    sample = X_test_scaled.loc[idx:idx]
    prediction = best_model.predict(sample)[0]
    probability = best_model.predict_proba(sample)[0]
    actual = y_test.loc[idx]
    
    risk_label = "HIGH RISK" if prediction == 1 else "LOW RISK"
    actual_label = "HIGH RISK" if actual == 1 else "LOW RISK"
    
    print(f"\nSample {idx}:")
    print(f"  Predicted: {risk_label} (Probability: {probability[1]:.2%})")
    print(f"  Actual:    {actual_label}")
    print(f"  Correct:   {'✅ Yes' if prediction == actual else '❌ No'}")

print("\n" + "="*80)

## 10. Save the Trained Model

Save the best performing model for deployment in the Flask API.

In [ ]:
# Save the trained model
model_path = 'models/women_risk_model.pkl'
joblib.dump(best_model, model_path)

print("="*60)
print("MODEL SAVED SUCCESSFULLY")
print("="*60)
print(f"\n✅ Model: {best_model_name}")
print(f"✅ Saved to: {model_path}")
print(f"✅ Scaler saved to: models/scaler.pkl")
print(f"✅ Label encoders saved to: models/label_encoders.pkl")
print("\n" + "="*60)

# Create model info file
model_info = f"""
Women Harassment Risk Predictor - Model Information
====================================================

Model Type: {best_model_name}
Training Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

Dataset Information:
- Total Samples: {len(df)}
- Training Samples: {len(X_train)}
- Testing Samples: {len(X_test)}
- Number of Features: {X_train.shape[1]}

Model Performance (Test Set):
- Accuracy:  {accuracy:.4f}
- Precision: {precision:.4f}
- Recall:    {recall:.4f}
- F1-Score:  {f1:.4f}
- ROC-AUC:   {roc_auc:.4f}

Files Generated:
- models/women_risk_model.pkl (trained model)
- models/scaler.pkl (feature scaler)
- models/label_encoders.pkl (categorical encoders)
"""

with open('models/model_info.txt', 'w') as f:
    f.write(model_info)

print("✅ Model information saved to: models/model_info.txt")

## ✅ Training Complete!

The model has been trained successfully and saved. You can now:

1. **Start the Flask API server:**
   ```bash
   python app.py
   ```

2. **Access the dashboard:**
   - Open browser: http://127.0.0.1:5000/dashboard
   
3. **Test predictions:**
   - Fill in the questionnaire form
   - Get real-time risk predictions
   
4. **View statistics:**
   - Dashboard shows total responses
   - Risk level distribution
   - High risk cases count

### Next Steps:
- Review model performance metrics above
- Test the API with different inputs
- Monitor prediction accuracy
- Retrain model with more data as needed